# Ordering and simulations (bounded model checking)

In each of the examples so far we have looked at systems that are effectively static in time. We
don't consider how entities change over time, just their relationships with one another. This is 
useful in some situations but, particularly in biology, we want to know how systems develop over 
time. Here we will start to look at some examples where we describe this type of system.

This can be referred to as bounded model checking; we look for solutions up to a bound (a number
of steps taken). Our solutions are therefore restricted to the bound; sometimes this is fine, but 
in others it can be a limitation. Increasing the bound makes the solutions slower and harder to 
find as the size of state space increases. In a worst case scenario an increase of a single step
may transform the problem from one that can be solved in ms to one that takes years! There may 
therefore be a largest theoretical bound that is greater than the bound that can practically be 
tested, and you should be aware that the results may not hold for larger, untestable bounds.

## Getting started

In this section we will be using Z3 as a library. To start we will need to download the files if they are not already available. These first cells download Z3 as a zip, extract it, and load it into memory. We then reference the extracted file and open it as a module.

In [3]:
#r "System.IO.Compression.FileSystem.dll"

open System
open System.IO
open System.IO.Compression
open System.Net
//Specify Tls version to avoid cryptic connection errors
System.Net.ServicePointManager.SecurityProtocol <- SecurityProtocolType.Tls12 ||| SecurityProtocolType.Tls11 
 
let wc = new WebClient()

type OS =
        | OSX            
        | Windows
        | Linux

let getOS = 
        match int Environment.OSVersion.Platform with
        | 4 | 128 -> Linux
        | 6       -> OSX
        | _       -> Windows

if  true <> System.IO.File.Exists("z3/LICENSE.txt") then 
    match getOS with
    | Linux ->  wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-ubuntu-16.04.zip", @"z3.zip")
                //This will take a while
                ZipFile.ExtractToDirectory("z3.zip", ".") 
                System.IO.Directory.Move("z3-4.6.0-x64-ubuntu-16.04","z3")
    | Windows ->wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-win.zip", @"z3.zip")
                //This will take a while
                ZipFile.ExtractToDirectory("z3.zip", ".") 
                System.IO.Directory.Move("z3-4.6.0-x64-win","z3")

    | _ -> ()

In [ ]:
open System.Net
 = SecurityProtocolType.Tls11| SecurityProtocolType.Tls12;

In [2]:
#r "z3/bin/Microsoft.Z3.dll"

In [3]:
open Microsoft.Z3

## Die Hard with a Vengance

You have a 3 litre jug and a 5 litre jug, and need to measure 4 litres of water. You can empty 
the jugs onto the ground and into each other, and you can fill them from each other and the tap. 
Without measuring the volumes explicitly, how do you get 4 litres?

Now we have variables that change with time. The way we create them is not different from before
but we need to consider the initial state and the relationships between timepoints. 

In previous examples where variables did not change we created constants with the name of the 
variable; now we will add the time explictly to the variable name. So in the initial case we 
will have just two variables "Five-0" and "Three-0". Consistant naming is important so that we 
can encode the behaviour in a loop; other times will be written as "Five-%t" whtere %t is the time.

In [1]:
let assertUpdate (ctx:Context) (s:Solver) t t' = 
    //Convienience integers
    let zZero = ctx.MkInt(0)
    let zTwo = ctx.MkInt(2)
    let zFour = ctx.MkInt(4)
    let zFive = ctx.MkInt(5)
    let zThree = ctx.MkInt(3)

    //Create the variables
    let fiveState = ctx.MkIntConst(sprintf "Five-%d" t)
    let threeState = ctx.MkIntConst(sprintf "Three-%d" t)
    let fiveState' = ctx.MkIntConst(sprintf "Five-%d" t')
    let threeState' = ctx.MkIntConst(sprintf "Three-%d" t')

    //Simple updates; do nothing, fill from tap, empty to ground
    let doNothingFive = ctx.MkEq(fiveState,fiveState')
    let doNothingThree = ctx.MkEq(threeState,threeState')
    let fillFive = ctx.MkEq(fiveState',zFive)
    let fillThree = ctx.MkEq(threeState',zThree)
    let emptyFive = ctx.MkEq(fiveState',zZero)
    let emptyThree = ctx.MkEq(threeState',zZero)

    //Complex updates; fill three from five, fill five from three
    //You can transfer only if one jug ends up full or empty
    let transfer = ctx.MkEq(ctx.MkAdd(fiveState,threeState),ctx.MkAdd(fiveState',threeState'))

    //List all of the possible updates, turn them into constraints, add them to the solver
    let possibleUpdates = [|
                            ctx.MkAnd(doNothingFive,fillThree)
                            ctx.MkAnd(doNothingFive,emptyThree)
                            ctx.MkAnd(fillFive,doNothingThree)
                            ctx.MkAnd(emptyFive,doNothingThree)
                            ctx.MkAnd(transfer,emptyFive)
                            ctx.MkAnd(transfer,emptyThree)
                            ctx.MkAnd(transfer,fillFive)
                            ctx.MkAnd(transfer,fillThree)
                            |]

    let constraints = ctx.MkOr(possibleUpdates)
    s.Add(constraints)

The type 'Context' is not defined.
The type 'Solver' is not defined.
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
Lookup on object of

## Initialising the model- assertUpdate

We then initialise the model, by specifying how full the jugs are initially. Each jug starts off empty
so we set the variables "Five-0" and "Three-0" to be equal to zero. 

We then define the transitions that the jugs can make according to the actions we can perform. 
This is done in the step function that asserts how the jugs update between two times, and the 
bounds of the jugs (i.e. the total amount of water they can hold). The update itself is specified
in assertUpdate. We can do a limited number of things;

* Empty each of the jugs (*emptyThree,emptyFive*)
* Do nothing to each of the jugs (*doNothingThree,doNothingFive*)
* Fill each of the jugs (*fillThree,fillFive*)
* Transfer fluid from one jug to the other, leading to either one jug being filled or one emptied (*transfer*)

In the last case the total volume of water must stay the same, and one of the jugs must be either
emptied or filled. We can then add all of the different options to an "Or" expression, and add this 
as a constraint.

## Additional constraints- assertBounds

Arguably we don't need it, but *assertBounds* ensures that the jugs stay in their defined limits. This may prevent certain bugs

In [5]:
let assertBounds (ctx:Context) (s:Solver) t =
    //Convienience integers
    let zZero = ctx.MkInt(0)
    let zFive = ctx.MkInt(5)
    let zThree = ctx.MkInt(3)
    //Create the variables
    let fiveState = ctx.MkIntConst(sprintf "Five-%d" t)
    let threeState = ctx.MkIntConst(sprintf "Three-%d" t)

    let constraints = ctx.MkAnd([|
                                    ctx.MkGe(fiveState,zZero)
                                    ctx.MkLe(fiveState,zFive)
                                    ctx.MkGe(threeState,zZero)
                                    ctx.MkLe(threeState,zThree)
                                    |])
    s.Add(constraints)

Finally we define functions that tie these together. Step asserts that between two given times, t and t', an action is taken

In [6]:
let step ctx s t t' =
    assertUpdate ctx s t t'
    assertBounds ctx s t
    assertBounds ctx s t'

*setState* is a convienince function that allows us to specify the state of a system at a given time. This is important for defining the initial state, and the final state.

In [7]:
let setState (ctx:Context) (s:Solver) t (three:int) (five:int) = 
    //Convienience integers
    let zThree = ctx.MkInt(three)
    let zFive = ctx.MkInt(five)
    //Create the variables
    let fiveState = ctx.MkIntConst(sprintf "Five-%d" t)
    let threeState = ctx.MkIntConst(sprintf "Three-%d" t)
    let constraints = ctx.MkAnd([|
                                    ctx.MkEq(fiveState,zFive)
                                    ctx.MkEq(threeState,zThree)
                                |])
    s.Add(constraints)

let initial ctx s t =
    setState ctx s t 0 0
let final (ctx:Context) (s:Solver) t = 
    let zFour = ctx.MkInt(4)
    let fiveState = ctx.MkIntConst(sprintf "Five-%d" t)
    s.Add(ctx.MkEq(fiveState,zFour))

To test different bounds we then then use a loop and add a new step for each turn of the loop, testing
at each stage for a solution. If we run the main function with a maximum bound of 10 we can find a 
solution quickly, in only 6 steps! 

In [8]:
let main maxBound = 
    let ctx = new Context()
    let s = ctx.MkSolver()
    initial ctx s 0
    let rec core i =
        if i = maxBound then printf "No results within bound of %d\n" maxBound else 
            step ctx s (i-1) i
            s.Push()
            final ctx s i
            let r = s.Check()
            match r with 
            | Status.UNSATISFIABLE ->
                s.Pop()
                printf "Unsat- No answer with a bound of %d\n" i
                core (i+1)
            | Status.SATISFIABLE ->
                s.Pop()
                printf "Sat- Got a result at bound %d\n" i
                printf "3Jug\t5Jug\n"
                for t=0 to i do
                    let threeState = s.Model.ConstInterp(ctx.MkIntConst(sprintf "Three-%d" t))
                    let fiveState = s.Model.ConstInterp(ctx.MkIntConst(sprintf "Five-%d" t))
                    printf "%O\t%O\n" threeState fiveState
            | _ -> failwith "Unknown response from Z3"
    core 1

In [9]:
main 10

Unsat- No answer with a bound of 1
Unsat- No answer with a bound of 2
Unsat- No answer with a bound of 3
Unsat- No answer with a bound of 4
Unsat- No answer with a bound of 5
Sat- Got a result at bound 6
3Jug	5Jug
0	0
0	5
3	2
0	2
2	0
2	5
3	4


## Exercises

1. Now imagine that some updates are not allowed; for example, you couldn't empty the 3 litre jug without transferring the contents to the other jug. What happens to the solution if you prevent those from occuring? Modify the code to find out
2. Within a small bound, are there any update types you must have?